### 스마일게이트 UnsmileData Pretrained model 사용해보기

참고자료: https://github.com/smilegate-ai/korean_unsmile_dataset

In [2]:
from datasets import load_dataset
datasets = load_dataset('smilegate-ai/kor_unsmile')

Using custom data configuration smilegate-ai--kor_unsmile-1dba960877497f9f
Reusing dataset parquet (C:\Users\Erin\.cache\huggingface\datasets\parquet\smilegate-ai--kor_unsmile-1dba960877497f9f\0.0.0\0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)
100%|██████████| 2/2 [00:00<00:00, 186.45it/s]


In [3]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 15005
    })
    valid: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 3737
    })
})


실제 훈련된 모델은 '개인지칭'과 'label'은 빠져있음.
'개인지칭'의 경우 추가 정보(개인을 지칠할 경우 모욕적일 수도 있어서...?)

In [4]:
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer
model_name = 'smilegate-ai/kor_unsmile'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = TextClassificationPipeline(
        model = model,
        tokenizer = tokenizer,
        device = 0,   # cpu: -1, gpu: gpu number
        return_all_scores = True,
        function_to_apply = 'sigmoid'
    )


In [28]:
# 예시 문장
for result in pipe("남자는 집가서 애나 봐라")[0]:
    print(result)

{'label': '여성/가족', 'score': 0.388251394033432}
{'label': '남성', 'score': 0.1427825689315796}
{'label': '성소수자', 'score': 0.010228877887129784}
{'label': '인종/국적', 'score': 0.01514759287238121}
{'label': '연령', 'score': 0.008011103607714176}
{'label': '지역', 'score': 0.009965253062546253}
{'label': '종교', 'score': 0.00785824190825224}
{'label': '기타 혐오', 'score': 0.009737115353345871}
{'label': '악플/욕설', 'score': 0.1703701913356781}
{'label': 'clean', 'score': 0.20860446989536285}


갖고 있는 데이터로 테스트

In [25]:
import numpy as np
import pandas as pd

In [26]:
df = pd.read_csv('./소신투표/소신투표_result.csv', encoding='utf-8')

In [27]:
result = []
for t in df['text'].astype(str):
    result.extend(pipe(t))

In [29]:
# 결과 예시
result[0]

[{'label': '여성/가족', 'score': 0.01607031561434269},
 {'label': '남성', 'score': 0.003696973202750087},
 {'label': '성소수자', 'score': 0.006438276264816523},
 {'label': '인종/국적', 'score': 0.020544659346342087},
 {'label': '연령', 'score': 0.007808662950992584},
 {'label': '지역', 'score': 0.030384093523025513},
 {'label': '종교', 'score': 0.010064619593322277},
 {'label': '기타 혐오', 'score': 0.03580975532531738},
 {'label': '악플/욕설', 'score': 0.21212667226791382},
 {'label': 'clean', 'score': 0.3380521237850189}]

In [31]:
score = [[item['score'] for item in i] for i in result]

In [32]:
score = np.round(score,2)

In [34]:
label =  ['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean']
new_df = pd.DataFrame(score, columns=label)

In [36]:
# df에 text열 추가
new_df.insert(0, 'text', df['text']) # 앞에 숫자가 순서

In [37]:
new_df.head()

,text,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean
0,문프님 말도 안 듣는 문파 오소리들이 이낙연대표님 말인들 듣겠냐 아닌건 아닌거지 소...,0.02,0.00,0.01,0.02,0.01,0.03,0.01,0.04,0.21,0.34
1,내가 솔직히 대선보고 나서 그냥 심 뽑는 소신 투표할까 ㅇㅈㄹ했었는데 윤은 나의 망...,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.79,0.15
2,이거 뽑으나 저거 뽑으나 다 사표나 마찬가진데 왜 자꾸 무조건 양자택일해야한다고 하...,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.25,0.71
3,저새끼들 오냐오냐 하니까 매일같이 실은 내가 더 나쁜새낀데ㅋ 이 대결하고 자빠져있는...,0.02,0.02,0.01,0.02,0.01,0.01,0.01,0.03,0.89,0.08
4,역설적으로 소신투표 해야 15년 뒤 이준석 김남국의 대선을 안본다고 생각한다,0.02,0.01,0.02,0.02,0.01,0.02,0.01,0.01,0.06,0.91


In [38]:
result.to_csv("./소신투표/소신투표_unsmile2.csv", encoding='utf-8-sig',sep=',', index=False)